In [1]:
%load_ext sql

There's a new jupysql version available (0.10.9), you're running 0.10.8. To upgrade: pip install jupysql --upgrade
Deploy AI and data apps for free on Ploomber Cloud! Learn more: https://docs.cloud.ploomber.io/en/latest/quickstart/signup.html


Loading configurations from /home/hugo/Documents/skola/edaf75/repo/pyproject.toml.

Settings changed:

Config,value
feedback,True
autopandas,True


In [2]:
%sql sqlite:///movies.sqlite

Connecting to 'sqlite:///movies.sqlite'

In [8]:
%%sql
insert
into   tickets(user_name, performance_id)
values ('bob', '5fa0059c1b18646d54cf4204d2ac6109')
returning ticket_id

Running query in 'sqlite:///movies.sqlite'

,ticket_id
0,7cf6185e18ebc1a7c65d843f1ac65ec6


In [10]:
%%sql
select *
from tickets

Running query in 'sqlite:///movies.sqlite'

,ticket_id,user_name,performance_id
0,3edf76d8a2700f4bc54dc8949aee353b,bob,5fa0059c1b18646d54cf4204d2ac6109
1,7cf6185e18ebc1a7c65d843f1ac65ec6,bob,5fa0059c1b18646d54cf4204d2ac6109


In [28]:
%%sql
with taken_seats(performance_id, taken) as (
    select performance_id, count() as taken
    from performances
            join theaters using(theater_name)
            join tickets using(performance_id)
    group by performance_id
)
select performance_id, date, start_time, title, year, theater_name, capacity - coalesce(taken, 0) as remaining_seats
from performances
        join theaters using(theater_name)
        join movies using(imdb)
     left outer join taken_seats using(performance_id)

Running query in 'sqlite:///movies.sqlite'

,performance_id,date,start_time,title,year,theater_name,remaining_seats
0,5fa0059c1b18646d54cf4204d2ac6109,2021-02-22,19:00,TheDig,2021,Regal,14
1,fea61d39de2410b6d14a0481166ef26e,2021-02-22,21:00,TheDig,2021,Regal,16
2,7e6b684bcc6a056689e88b1d3ec6d9f7,2021-02-22,19:00,TheLittleThings,2021,Skandia,100
